In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import numpy as np
import json
import os
import urllib

from imageai.Classification import ImageClassification

from PIL import Image
import requests

pd.set_option('display.max_columns', None)

from tqdm import tqdm
tqdm.pandas()

from datetime import datetime

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'horego-big-query'

# Make clients.
horego_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

In [3]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
import gspread as gs
gc = gs.service_account(filename='Horego-GCP-Keys.json')
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1oh-vUpzaj6MfKmcuYiordlS2HNd1ATuSDNqS2JNEtkI/edit?usp=sharing')

worksheet = sheet.worksheet('Sheet1')
data = pd.DataFrame(worksheet.get_all_values())
data.columns=data.iloc[0]
data=data[1:]
data.columns=['date', 'Selected Date', 'place_id', 'photo_id', 'tags']
# data['date']=datetime.now().strftime('%Y-%m-%d')
data.tags=data.tags.map({'MAIN_BANNER':'main_banner','BANNER':'priority_banner'})
data.head()

,date,Selected Date,place_id,photo_id,tags
1,18/10/23,12/10/23 10.46,ChIJ8WJQonXvaC4RyAYDcRpOZOE,AF1QipPSUVTwcigJVHC-CUyaarcoaSZOw_kANuGbKd4V,main_banner
2,18/10/23,12/10/23 10.46,ChIJ8WJQonXvaC4RyAYDcRpOZOE,AF1QipOvVdFrl6BG5sy2ftTzcUV_vfGnIJMBIk_7MORw,priority_banner
3,18/10/23,12/10/23 10.46,ChIJ8WJQonXvaC4RyAYDcRpOZOE,AF1QipOEBIH6apUazxW1BzuiLTpTdFL3F5Tw7KMvW70D,priority_banner
4,18/10/23,12/10/23 10.46,ChIJ8WJQonXvaC4RyAYDcRpOZOE,AF1QipMMgHzQ7WdLdX2lkGARuwr3MsNeTqZcgbllOPzn,priority_banner
5,18/10/23,12/10/23 10.46,ChIJ8WJQonXvaC4RyAYDcRpOZOE,AF1QipMd44KU4f7qxcBC65ORViNFvupyLIJbXua5Qvk,priority_banner


In [5]:
print(data['date'].unique())

['18/10/23' '23/10/23' '30/10/23']


In [6]:
data=data[data['date']=='30/10/23']
data['date']=datetime.now().strftime('%Y-%m-%d')

In [7]:
%%time

query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.ops_choosen_photos` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1
CPU times: user 77.9 ms, sys: 20.5 ms, total: 98.4 ms
Wall time: 2.48 s


In [8]:
my_schema=[]
save_columns=[]
for column in df_temp.columns :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].to_csv('results/choosen_photos.csv',sep=';',index=False)

In [9]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

# df_temp=pd.read_csv('results/choosen_photos.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     my_schema.append(bigquery.SchemaField(column, "STRING"))

src_path_file='results/choosen_photos.csv'
target_path_file='scrapping_results/choosen_photos.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "ops_choosen_photos"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/choosen_photos.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

# table_schema = "public_datalake"
# table_name = "textract_menu_ocr_results"
# gcs_path_uri = f"gs://{bucket_name}/scrapping_results/clear_raw_table.csv"
# create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema)

print('')
#############################################################################################

src_path_file='results/choosen_photos.csv'
target_path_file='scrapping_results/choosen_photos.csv'
bucket_name='horego-bq'
upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

table_schema = "horego_datalake_dev"
table_name = "ops_choosen_photos"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/choosen_photos.csv"
create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/choosen_photos.csv
Path in GCS: cashbac_datalake/scrapping_results/choosen_photos.csv
table on : cashbac_datalake_prod.ops_choosen_photos

Uploading results/choosen_photos.csv
Path in GCS: horego-bq/scrapping_results/choosen_photos.csv
table on : horego_datalake_dev.ops_choosen_photos


### insert into list table insert 

In [10]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_insert_to_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_insert_to_prod.csv',sep=';',index=False)

5
CPU times: user 34.9 ms, sys: 17.4 ms, total: 52.2 ms
Wall time: 822 ms


In [11]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

# df_temp=pd.read_csv('results/choosen_photos.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     my_schema.append(bigquery.SchemaField(column, "STRING"))

src_path_file='results/input_list_insert_to_prod.csv'
target_path_file='scrapping_results/input_list_insert_to_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_insert_to_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_insert_to_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_insert_to_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_insert_to_prod.csv
table on : cashbac_datalake_prod.list_insert_to_prod
